In [ ]:
! git clone https://github.com/huggingface/diffusers.git

In [1]:
import torch
import matplotlib.pyplot as plt
import random
import threading
from librosa.beat import beat_track
from pythonosc import dispatcher, osc_server
import numpy as np
from datasets import load_dataset
from diffusers_local import DiffusionPipeline, DDIMScheduler, AudioDiffusionPipeline
from diffusers_local import UNet2DModel as UNet2DModel_local
import soundfile as sf
from CreateDataset import AudioProcessor
from NetworkBending import NetworkBending
import time

osc_lock = threading.Lock()

NB = NetworkBending()

device = "cuda" if torch.cuda.is_available() else "cpu"
audio_diffusion = DiffusionPipeline.from_pretrained("teticio/audio-diffusion-ddim-256")
# apply the custom unet model so we can Networkbend the audio
audio_diffusion.unet = UNet2DModel_local.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="unet", network_bending=NB)

#set the scheduler to DDIM
audio_diffusion.scheduler = DDIMScheduler.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="scheduler")
audio_diffusion.to(device)

#ds = load_dataset('teticio/audio-diffusion-256')
#generator=torch.Generator(device="cpu").manual_seed(42)

alpha = 0
latent1 = 0
latent2 = 1
diffusion_steps = 10
make_loop = 0
generate = 1

c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]An error occurred while trying to fetch C:\Users\danhearn\.cache\huggingface\hub\models--teticio--audio-dif

ValueError: Pipeline <class 'diffusers_local.pipelines.deprecated.audio_diffusion.pipeline_audio_diffusion.AudioDiffusionPipeline'> expected {'vqvae', 'mel', 'unet', 'scheduler', 'vocoder'}, but only {'scheduler', 'mel', 'unet'} were passed.

In [1]:
import torch
import matplotlib.pyplot as plt
import random
import threading
from librosa.beat import beat_track
from pythonosc import dispatcher, osc_server
import numpy as np
from datasets import load_dataset
from diffusers_local import DDIMScheduler, AudioDiffusionPipeline, UNet2DModel as UNet2DModel_local, Mel
import soundfile as sf
from CreateDataset import AudioProcessor
from NetworkBending import NetworkBending
from MelVocoder import MelVocoder  
from HifiVocoder import HifiVocoder
import time

osc_lock = threading.Lock()

NB = NetworkBending()
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the MelVocoder for audio synthesis
vocoder = HifiVocoder(device=device, checkpoint_file='hifigan\hifi256\g_00280000')  # Use Vocoder for spectrogram-to-audio conversion

# Load the custom UNet model and apply NetworkBending
unet = UNet2DModel_local.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="unet", network_bending=NB)
# Set up the DDIM scheduler
scheduler = DDIMScheduler.from_pretrained("teticio/audio-diffusion-ddim-256", subfolder="scheduler")

mel = Mel()

# Initialize the AudioDiffusionPipeline with custom components
audio_diffusion = AudioDiffusionPipeline(
    vqvae=None,  # Assuming no VQ-VAE is needed; adjust if required
    unet=unet,
    mel=mel,
    vocoder=vocoder,
    scheduler=scheduler,
)
audio_diffusion.to(device)

# Optional: Load a dataset
# ds = load_dataset('teticio/audio-diffusion-256')
# generator = torch.Generator(device="cpu").manual_seed(42)

print(mel)

alpha = 0
latent1 = 0
latent2 = 1
diffusion_steps = 10
make_loop = 0
generate = 1
save = 0

c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
c:\Users\danhearn\.conda\envs\diffusertrack\Lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn

Loading 'hifigan\hifi256\g_00280000'
Complete.
Removing weight norm...


c:\Users\danhearn\Documents\GitHub\diffusertrack\HifiVocoder.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(self.checkpoint_file, map_loca

Mel {
  "_class_name": "Mel",
  "_diffusers_version": "0.31.0.dev0",
  "hop_length": 512,
  "n_fft": 2048,
  "n_iter": 32,
  "sample_rate": 22050,
  "top_db": 80,
  "x_res": 256,
  "y_res": 256
}



In [3]:
from MelVocoder import MelVocoder

# Initialize the vocoder
vocoder = MelVocoder(device_name="cuda", model_name="best_netG.pt")

# Vocode a sample (assuming `sample` is a Mel spectrogram tensor)
#audio_output = vocoder.vocode(sample)

In [2]:
# Load audio and process it into 5 second spectrograms
input_dir = r"data\\Floating Points - 'Key103' (Official Video)_chunks" 
processor = AudioProcessor(input_dir)
processor.process_directory()

c:\Users\danhearn\Documents\GitHub\diffusertrack\MelVocoder.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vocoder.load_state_dict(torch.load(vocoder_param_fp, map_loc

Saved mel spectrogram: chunk_1_225084ef-3ee8-4c36-92e1-571ae5627e43.png
Saved mel spectrogram: chunk_1_5aa4c38d-083a-4bcb-8afe-cafd77f6634a.png
Saved mel spectrogram: chunk_1_8e1ecc20-cc9a-497e-a6bb-5d20c3f01376.png
Saved mel spectrogram: chunk_1_140a5bf5-b432-4589-9885-540c0154b62f.png
Saved mel spectrogram: chunk_1_7c4f4542-e4fb-4f8e-b996-6c90a29a67b4.png
Saved mel spectrogram: chunk_1_c565a117-cd68-4a63-89d0-7f73a66cdf43.png
Saved mel spectrogram: chunk_1_c0eda7cc-d20e-41f7-8f46-8372ce73e1f3.png
Saved mel spectrogram: chunk_1_65dae36b-4625-4f2a-810d-d375fb9d24b9.png
Saved mel spectrogram: chunk_1_ccc53e98-5a94-48d7-b385-0c6f999d0d7d.png
Saved mel spectrogram: chunk_1_d5edbfff-5226-4f48-98a7-28219069e359.png
Saved mel spectrogram: chunk_1_304ecfa1-ec4c-4ad9-a33c-014d161c463b.png
Saved mel spectrogram: chunk_1_4a43d775-82c2-4f60-994b-1bf2d9b48cb6.png
Saved mel spectrogram: chunk_1_56f96a79-40d4-43be-8980-b975be270790.png
Saved mel spectrogram: chunk_1_b759bda4-7fb4-4fac-be5f-f2cabc4a8

In [2]:
def loop_it(audio: np.ndarray,
        sample_rate: int,
        loops: int = 1) -> np.ndarray:
    """Loop audio

    Args:
        audio (np.ndarray): audio as numpy array
        sample_rate (int): sample rate of audio
        loops (int): number of times to loop

    Returns:
        (float, np.ndarray): sample rate and raw audio or None
    """
    _, beats = beat_track(y=audio, sr=sample_rate, units='samples')
    for beats_in_bar in [16, 12, 8, 4]:
        if len(beats) > beats_in_bar:
            return np.tile(audio[beats[0]:beats[beats_in_bar]], loops)
    return None

def interpolation(address, *args):
    
    """Interpolate between two latents for generation
    Args:
        address (str): OSC address
        *args: OSC arguments
    """

    global alpha, latent1, latent2
    
    if address == "/alpha":
        alpha = args[0]
    if address == "/latent1" and latent2 != args[0]:
        latent1 = int(args[0])
    if address == "/latent2" and latent1 != args[0]:
        latent2 = int(args[0])

def inference(address, *args):

    """Control the inference process
    Args:
        address (str): OSC address
        *args: OSC arguments
    """

    global diffusion_steps, make_loop, generate

    if address == "/steps":
        diffusion_steps = int(args[0])
    if address == "/loop":
        make_loop = int(args[0])
    if address == "/generate":
        generate = int(args[0])
    if address == "/save":
        save = int(args[0])

In [3]:
#load 20 images from the default dataset
ds = load_dataset('teticio/audio-diffusion-256')
images = []

for i in range(2):
    images.append(random.choice(ds['train'])['image'])

images[0].show()


In [3]:
import os
from PIL import Image
import torch
import numpy as np

# Assuming `audio_diffusion` is your Diffusion model pipeline
encoded_images = []

# Directory containing mel spectrograms
spectrogram_dir = "data/mel_spectrograms"

# Get all file names in the directory
file_names = os.listdir(spectrogram_dir)

# Pick 20 random indexes
random_indexes = random.sample(range(len(file_names)), 10)

# Loop through each random index
for i in random_indexes:
    spectrogram_path = os.path.join(spectrogram_dir, file_names[i])
    spectrogram_image = Image.open(spectrogram_path).convert("L")
    encoded_image = audio_diffusion.encode([spectrogram_image], steps=100)
    encoded_images.append(encoded_image)



  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\danhearn\Documents\GitHub\diffusertrack\diffusers_local\models\attention_processor.py:2367: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  hidden_states = F.scaled_dot_product_attention(
100%|██████████| 100/100 [00:02<00:00, 41.00it/s]


In [4]:
for encoded_image in encoded_images:
    plt.imshow(encoded_image.cpu().squeeze(), cmap='gray')
    plt.axis('off')  # Hide axes
    plt.show()
    

C:\Users\danhearn\AppData\Local\Temp\ipykernel_26772\1570746484.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [4]:
#osc listener
from pythonosc import dispatcher, osc_server, udp_client

ip="127.0.0.1"
port_receive=9999
port_send = 1111

d = dispatcher.Dispatcher()
# Map incoming OSC messages to the osc_receive method
d.map("/rotate_x_radian",  NB.osc_receive)
d.map("/rotate_y_radian", NB.osc_receive)
d.map("/rotate_z_radian", NB.osc_receive)
d.map("/rotate_x_scaling_factor", NB.osc_receive)
d.map("/rotate_y_scaling_factor", NB.osc_receive)
d.map("/rotate_z_scaling_factor", NB.osc_receive)
d.map("/scale_factor", NB.osc_receive)
d.map("/layer", NB.osc_receive)
d.map("/scale", NB.osc_receive)
d.map("/reflect", NB.osc_receive)
d.map("/erosion", NB.osc_receive)
d.map("/dilation", NB.osc_receive)
d.map("/gradient", NB.osc_receive)
d.map("/sobel", NB.osc_receive)
d.map("/add_rand_rows", NB.osc_receive)
d.map("/normalize", NB.osc_receive)
d.map("/rotate_x", NB.osc_receive)
d.map("/rotate_y", NB.osc_receive)
d.map("/rotate_z", NB.osc_receive)
d.map("/alpha", interpolation)
d.map("/latent1", interpolation)
d.map("/latent2", interpolation)
d.map("/steps", inference)
d.map("/loop", inference)
d.map("/generate", inference)
d.map("/save", inference)

s = osc_server.ThreadingOSCUDPServer((ip, port_receive), d)

send = udp_client.SimpleUDPClient(ip, port_send)

def send_osc(address, value):
    """Send OSC message
    Args:
        address (str): OSC address
        value (float): OSC value
    """
    send.send_message(address, value)


osc_thread = threading.Thread(target=s.serve_forever)
osc_thread.daemon = True # This will allow the main program to exit even if the OSC server is still running
osc_thread.start()
print("OSC server started - listening on port 9999")

OSC server started - listening on port 9999


In [5]:
# Main loop for image generation
try:
    while True:
        step = 0
        with osc_lock: 

            if generate == 1:
             
                output = audio_diffusion(steps=diffusion_steps,
                noise=AudioDiffusionPipeline.slerp(encoded_images[latent1], encoded_images[latent2], alpha), eta=0)

                output.images[0].save("generated_image.png")
                audio = output.audios[0, 0]

                if save == 1:
                    sf.write(f"results/audio/layer_{NB.layer_selection}_output_audio.wav", audio, audio_diffusion.mel.get_sample_rate())
                    output.images[0].save(f"results/audio/layer_{NB.layer_selection}_generated_image_{step}.png")
                    step += 1
                    save = 0

                if make_loop == 1:
                    loop = loop_it(audio, audio_diffusion.mel.get_sample_rate())
                    sf.write("output_audio.wav", loop, audio_diffusion.mel.get_sample_rate())
                elif make_loop == 0:
                    #time.sleep(0.05)
                    sf.write("output_audio.wav", audio, audio_diffusion.mel.get_sample_rate())
                
                send_osc("/generated", 1)



except KeyboardInterrupt:
    s.shutdown()
    print("Terminating the loop gracefully.")

100%|██████████| 10/10 [00:00<00:00, 39.89it/s]


Sample shape adjusted to torch.Size([1, 256, 256])


100%|██████████| 10/10 [00:00<00:00, 41.41it/s]


Sample shape adjusted to torch.Size([1, 256, 256])


100%|██████████| 10/10 [00:00<00:00, 43.94it/s]


...

Sample shape adjusted to torch.Size([1, 256, 256])


100%|██████████| 999/999 [00:24<00:00, 41.09it/s]


Sample shape adjusted to torch.Size([1, 256, 256])


100%|██████████| 999/999 [00:24<00:00, 41.06it/s]


Sample shape adjusted to torch.Size([1, 256, 256])
Terminating the loop gracefully.
